In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.1f}'.format)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf",
                                          use_auth_token=True)

/Users/hamel/mambaforge/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
from common import questions
import numpy as np

lens = [len(tokenizer.encode(q)) for q in questions.questions]
print(lens)
print(np.mean(lens))

[15, 12, 10, 35, 11, 26, 12, 23, 60]
22.666666666666668


In [ ]:
files = ['vllm-2/bench-vllm-2.csv',
'bentoml/bench-bentoml.csv',
'sagemaker/bench-sagemaker-flashattn.csv',
'triton-tensorRT/bench-triton-tensorRT-llm.csv',
'triton-tensorRT-quantized/bench-triton-tensorRT-llm-quantized.csv',
'triton-vllm/bench-triton.csv',
'triton-vllm-awq/bench-triton-vllm-awq.csv',
'triton-tensorRT-quantized-awq/bench-triton-tensorRT-llm-quantized-awq.csv',
'anyscale/bench-anyscale.csv']

In [ ]:
df = pd.concat([pd.read_csv(f) for f in files])
result = df.groupby('note')[['tok_count', 'time']].mean().reset_index()

In [ ]:
df['Tok/Sec'] = df.tok_count / df.time
df

,tok_count,time,question,answer,note,Tok/Sec
0,144,2.4,Write a Rust function that performs binary exp...,Write a Rust function that performs binary exp...,vllm-fastapi-server,60.2
1,210,3.5,What are the differences between Javascript an...,What are the differences between Javascript an...,vllm-fastapi-server,59.3
2,235,3.5,Write a story in the style of James Joyce abou...,Write a story in the style of James Joyce abou...,vllm-fastapi-server,66.3
3,11,0.0,Who does Harry turn into a balloon?,Who does Harry turn into a balloon??,vllm-fastapi-server,508.2
4,226,3.5,Write a tale about a time-traveling historian ...,Write a tale about a time-traveling historian ...,vllm-fastapi-server,63.8
...,...,...,...,...,...,...
3,201,3.4,Who does Harry turn into a balloon?,"Ah, a most intriguing question, my dear fell...",anyscale,59.6
4,201,3.4,Write a tale about a time-traveling historian ...,"As a time-traveling historian, I have always...",anyscale,59.6
5,114,2.0,What is the product of 9 and 8?,"Ah, a question that requires a simple yet el...",anyscale,57.1
6,201,3.2,"If a train travels 120 kilometers in 2 hours, ...","Oh, goodness gracious, thank you ever so kin...",anyscale,62.9


In [ ]:
result['Tok/Sec'] = result.tok_count / result.time
result = result.rename(columns={'tok_count': 'Avg Tok Count', 
               'time': 'Avg Time'})
result = result.sort_values(by='Tok/Sec', ascending=True).reset_index(drop=True)

In [ ]:
def backend(note):
    if 'vllm' in note.lower():
        return 'vLLM'
    if 'sagemaker' in note.lower():
        return 'TGI'
    if note == 'triton-tensorRT-llm':
        return 'TensorRT-LLM'
    if note == 'triton-tensorRT-llm-quantized':
        return 'TensorRT-LLM'
    if note == 'triton-vllm-awq':
        return 'vLLM'
    if note == 'triton-tensorRT-llm-quantized-awq':
        return 'TensorRT-LLM'
    if note == 'anyscale':
        return 'Anyscale'

def quant(note):
    if note in ['vllm-fastapi-server', 'bentoml-vllm', 'triton-tensorRT-llm', 'triton-vllm']:
        return 'None'
    if note == 'triton-tensorRT-llm-quantized':
        return 'INT8 weight-only quantization'
    if note == 'triton-vllm-awq':
        return 'AWQ 4bit quantization'
    if note == 'triton-tensorRT-llm-quantized-awq':
        return 'AWQ 4bit quantization'
    if note == 'anyscale':
        return '?'

def frontend(note):
    if 'fastapi' in note.lower():
        return 'FastAPI'
    if 'triton' in note.lower():
        return 'Triton'
    if note == 'bentoml-vllm':
        return 'OpenLLM'
    if 'sagemaker' in note.lower():
        return 'SageMaker'
    if note == 'anyscale':
        return 'Anyscale'

result['Front End'] = result['note'].apply(frontend)
result['Backend'] = result['note'].apply(backend)
result['Quantization']= result['note'].apply(quant)
final = result[['Front End', 'Backend', 'Quantization', 'Avg Tok Count', 'Avg Time', 'Tok/Sec']]
final

,Front End,Backend,Quantization,Avg Tok Count,Avg Time,Tok/Sec
0,SageMaker,TGI,None,223.6,6.6,33.8
1,OpenLLM,vLLM,None,191.1,3.4,55.7
2,Triton,vLLM,None,223.6,3.7,60.0
3,Anyscale,Anyscale,?,190.0,3.1,61.6
4,FastAPI,vLLM,None,158.1,2.4,66.0
5,Triton,TensorRT-LLM,None,225.6,3.3,69.0
6,Triton,TensorRT-LLM,INT8 weight-only quantization,225.6,1.8,125.3
7,Triton,vLLM,AWQ 4bit quantization,223.6,1.6,142.2
8,Triton,TensorRT-LLM,AWQ 4bit quantization,225.6,1.1,205.5


In [ ]:
final

,Front End,Backend,Quantization,Avg Tok Count,Avg Time,Tok/Sec
0,SageMaker,TGI,None,223.6,6.6,33.8
1,OpenLLM,vLLM,None,191.1,3.4,55.7
2,Triton,vLLM,None,223.6,3.7,60.0
3,Anyscale,Anyscale,?,190.0,3.1,61.6
4,FastAPI,vLLM,None,158.1,2.4,66.0
5,Triton,TensorRT-LLM,None,225.6,3.3,69.0
6,Triton,TensorRT-LLM,INT8 weight-only quantization,225.6,1.8,125.3
7,Triton,vLLM,AWQ 4bit quantization,223.6,1.6,142.2
8,Triton,TensorRT-LLM,AWQ 4bit quantization,225.6,1.1,205.5
